# 美团爬虫
- 介绍：因为美团有强大的爬虫机制，而商超爬虫相对与美食爬虫需要多爬取一些信息，所以我分为五步进行数据的爬取
    1. 第一步是爬取每个商铺的 shopID ，
    2. 第二步是使用shopId 爬取每家商超的tagId，即分类标签，
    3. 第三步是使用前两个ID获取SKU信息
    4. 第四步是使用shopID爬取商超信息，这一步是检查爬虫最严格的一步。
    

- 爬虫方法：
    1. 使用chrome或Filddler抓取美团外卖页面的请求头数据，最好使用虚拟手机号登录
    2. 修改对应的url与headers，并设置合理的sleepTime,防止被检测
    3. 开始爬虫

以下为美团商超爬虫代码及每部分需要注意的坑。


# 一.爬取shopID

*爬取shopID一般不会触发反爬机制，触发是也仅为账号下线，在浏览器上重新登录即可继续爬虫，但需要注意设置经纬度数据的格式为保留6位小数点和记录中断的信息*

In [4]:
import random
import requests
import random
import time
import hashlib
import os
import pandas as pd

In [121]:
def save_shop(json):
    shop_info = r'C:\Users\Trine\Desktop\my_data\shenz_shopId.csv'
    
    for data in json['data']['shopList']:
        info = {}
        info['shopId'] = data['mtWmPoiId']
        info['shopName'] = data['shopName']
        #info[ 'statusDesc'] = info_json[ 'statusDesc']
        df = pd.DataFrame(info, index=[0])
        print(info)
        df.to_csv(shop_info, header=0,index=False,encoding='utf_8_sig',mode='a+')
        
        if json['data']['poiHasNextPage'] == True:
            global lat,lon
            shop_json = get_shopId(ses, lat, lon, json['data']['nextStartIndex'])
            if shop_json['msg'] == '成功':
                save_shop(shop_json)
        
def get_shopId(ses, Lat, Lon, page=0):
    user_agent_list = [
        "Mozilla/5.0 (compatible; MSIE 10.0; Windows Phone 8.0; Trident/6.0; IEMobile/10.0; ARM; Touch; NOKIA; Lumia 920) UCBrowser/10.1.0.563 Mobile",
    ]
    User_Agent = random.choice(user_agent_list)
    headers = {
                "Host": "i.waimai.meituan.com",
                "Cookie": "_lxsdk_cuid=17095e7b964c8-081b2ab2f44e58-5040231b-144000-17095e7b964c8; terminal=i; w_utmz=\"utm_campaign=(direct)&utm_source=5000&utm_medium=(none)&utm_content=(none)&utm_term=(none)\"; w_uuid=d3taEsYmWa5EMBZXi5nkJhrbMmEEcj2t20rktYn7P9vZzLdc286ICBCkZCURxXrv; iuuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; _lxsdk=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; webp=1; __utmz=74597006.1585277189.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; uuid=a36ec623ffdc4ef4b39f.1585293909.1.0.0; ci=20; cityname=%E5%B9%BF%E5%B7%9E; wm_order_channel=mtib; utm_source=60030; a2h=3; IJSESSIONID=1fmbae6ndhfc71b77zfzqfz5dk; __utma=74597006.2081068397.1585277189.1585535048.1585990931.4; __utmc=74597006; latlng=26.575229,107.964896,1585990931073; ci3=1; i_extend=H__a100001__b4; au_trace_key_net=default; openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; service-off=0; w_actual_lat=26576238; w_actual_lng=107977159; w_visitid=fc4c6006-a1f8-435c-aadd-d67a152587ac; w_latlng=23060548,113391350; _lx_utm=utm_source%3D60066; token=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; mt_c_token=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; oops=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; userId=131673809; cssVersion=8c43a663; w_token=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; _lxsdk_s=171531808b6-d6f-da7-e97%7C%7C14; channelType={%22mtib%22:%220%22}",
                "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
                "Origin": "https://h5.waimai.meituan.com",
                "Accept-Encoding": "br, gzip, deflate",
                "Accept": "application/json",
                "Accept-Language": "zh-cn,zh;q=0.9",
                "User-Agent": User_Agent,
            }

    headers_list = headers
    headers_list["Referer"] = "https://h5.waimai.meituan.com/waimai/mindex/kingkong?navigateType=101574&firstCategoryId=101574&secondCategoryId=101574&title=%E7%BE%8E%E5%9B%A2%E8%B6%85%E5%B8%82"

    formdata = {
                "navigateType": "101574",
                "firstCategoryId": "101574",
                
                "wm_latitude": str(Lat),
                "wm_longitude": str(Lon),
                "startIndex": str(page)
            }
    # 超市类的美团外卖链接
    url = "https://i.waimai.meituan.com/openh5/channel/kingkongshoplist?_=1587024517584&X-FOR-WITH=FXdeqbFbEHdtbrfMxUwOVnIdHKB2dewV0gsRUTzXSDnooztjKZhKkJAJHNZzZACCA%2BSO0wvVGmR3fS9z8tETjhV9Z3PjJQpbdFRW3%2FvPVC4g92QFa4RViHPDpqL%2BAkBxRAPL8G0D2ujYxxanLRmAIH0oQzkh%2BI5V99nGUYBsyVhTooUrjL9sxBqAnrs9lYtaQQH0N30SejQcjP70R6408g%3D%3D"

    proxies = {
                "http": "http://211.159.219.225:8118",
            }
    
    response = requests.post(url, data=formdata, headers=headers_list, stream=True)
    detail = response.json()
    return detail

In [134]:
if __name__ == '__main__':
    
    bus_info = pd.read_csv(r'C:\Users\Trine\Desktop\my_data\meituan\bus_shengzheng.csv')#使用深圳公交车站经纬度继续搜索
    
    for index, i in bus_info.iterrows():
        lat = int(round(i['lat'],6)*1000000)
        lon = int(round(i['lon'],6)*1000000)
        ses = requests.Session()
        shop_json = get_shopId(ses, lat, lon)
        
        if shop_json['msg'] == '成功':
            save_shop(shop_json)
        else:
            print('出现错误')#提示出现错误可能需要重新登录
            
        time.sleep(random.randint(1,2))

{'shopId': '1010757332402476', 'shopName': '美宜佳（茶光地铁店）'}
{'shopId': '1007377193146594', 'shopName': '鼎湖山泉，白金直营门店'}
{'shopId': '1071312076298985', 'shopName': '1987进口啤酒体验馆（红酒）'}
{'shopId': '997292609902778', 'shopName': '酒小二(南头店)'}
{'shopId': '1004907586930351', 'shopName': '全家便利店（大冲新城花园二店）'}
{'shopId': '979189322797592', 'shopName': '东北大板雪糕'}
{'shopId': '859102037200448', 'shopName': '酒仙网国际名酒城（新城花园店）'}
{'shopId': '1009975648349035', 'shopName': '711便利店(万科里7124店)'}
{'shopId': '1090291536806074', 'shopName': '十万大山桶装水'}
{'shopId': '979404071118478', 'shopName': '喜悦佳福（裕丰花园店）'}
{'shopId': '1062434378905821', 'shopName': '美宜佳（上川路店）'}
{'shopId': '975585845236171', 'shopName': '金松水业连锁配送（桶装水店）'}
{'shopId': '1012866161360294', 'shopName': '美宜佳（43区店）'}
{'shopId': '1013132449322472', 'shopName': '臣田水店（桶装水）'}
{'shopId': '1062975544773173', 'shopName': '美宜佳（14836上合店）'}
{'shopId': '1062434378905821', 'shopName': '美宜佳（上川路店）'}
{'shopId': '1098469154488603', 'shopName': 'GoBuy（茶光村店）'}
{'shopId': '951576

{'shopId': '943072942807555', 'shopName': '润之家便利店（新安云飞易点便利店）'}
{'shopId': '893886977333117', 'shopName': '福伴生活（固戍店）'}
{'shopId': '1090291536806074', 'shopName': '十万大山桶装水'}
{'shopId': '979404071118478', 'shopName': '喜悦佳福（裕丰花园店）'}
{'shopId': '1118273248731694', 'shopName': 'FUBAN福伴（玉湖湾花园店）'}
{'shopId': '1013132449322472', 'shopName': '臣田水店（桶装水）'}
{'shopId': '1042690414276535', 'shopName': '栗上皇（坪洲店）'}
{'shopId': '932868100484322', 'shopName': '711便利店(龙光世纪7176店)'}
{'shopId': '1051507982088928', 'shopName': '国喜酒庄'}
{'shopId': '1053999063166277', 'shopName': '金都名店品牌男装'}
{'shopId': '1041109866297314', 'shopName': '林记怡宝宝安观澜湖水店（坪洲店）'}
{'shopId': '914584424689735', 'shopName': '桶装水配送中心'}
{'shopId': '1056941115743796', 'shopName': '711便利店(海纳百川7149店)'}
{'shopId': '912179243041749', 'shopName': '酒小二(上川店)'}
{'shopId': '947672852766147', 'shopName': 'OK便利店'}
{'shopId': '1006007098547742', 'shopName': '天猫小店贤家便利店'}
{'shopId': '948441651915880', 'shopName': 'CHEERS齐饮进口葡萄酒(南山店)'}
{'shopId': '967760414776

{'shopId': '1120158739387176', 'shopName': '美宜佳(乐群社区菜市场店）'}
{'shopId': '985975371109604', 'shopName': '名创优品（深圳南山新城花园店）'}
{'shopId': '1018002942250050', 'shopName': '酒小二(流塘店)'}
{'shopId': '979077653657367', 'shopName': '德国啤酒专卖店'}
{'shopId': '857512899299138', 'shopName': '深通水业'}
{'shopId': '865162236013862', 'shopName': '美宜佳（国安居店）'}
{'shopId': '1125591873009390', 'shopName': '景田桶装水'}
{'shopId': '855081947767873', 'shopName': '十万大山桶装水（黄贝岭店）'}
{'shopId': '995918220391171', 'shopName': '酒加wine+前海葡萄酒体验店（海上世界店）'}
{'shopId': '860669700212534', 'shopName': '京东便利店（海湾店）'}
{'shopId': '1120566761280317', 'shopName': '良品铺子（深圳福田侨城坊店）'}
{'shopId': '870453635773703', 'shopName': '八马茶业（深圳华侨城沃尔玛店）'}
{'shopId': '980078381004188', 'shopName': '张新发（深圳龙华民治店）'}
{'shopId': '1071917666681859', 'shopName': '天福（石洲中路店）'}
{'shopId': '1080451766746776', 'shopName': '美宜多超市(下白石店)'}
{'shopId': '995815141183728', 'shopName': '有间酒馆'}
{'shopId': '1051752795231977', 'shopName': '明日文具'}
{'shopId': '1107626024799592', 'shop

{'shopId': '1061047104459549', 'shopName': '万宁（深圳花园城店）'}
{'shopId': '960862697345311', 'shopName': '美宜佳（南圆商业街店）'}
{'shopId': '871673406468785', 'shopName': '711便利店(东方银座0886店)'}
{'shopId': '980207230007438', 'shopName': '平阳烟酒茶行'}
{'shopId': '860721239874557', 'shopName': '金盛文具美术用品'}
{'shopId': '1090291536806074', 'shopName': '十万大山桶装水'}
{'shopId': '939572544467923', 'shopName': '711便利店(后海卓越7030店)'}
{'shopId': '947758752099141', 'shopName': 'TCYM便利店城市汇佳'}
{'shopId': '867185165622389', 'shopName': '澳洲1847酒庄'}
{'shopId': '1047079870847963', 'shopName': '美宜佳（南光粤17398店）'}
{'shopId': '1055536661417918', 'shopName': '美宜佳（西乡乐群社区店）'}
{'shopId': '932739251486610', 'shopName': '易站'}
{'shopId': '1075598453683937', 'shopName': '维客佳'}
{'shopId': '959960754214695', 'shopName': '易站（海城店）'}
{'shopId': '1021026599184230', 'shopName': '美宜佳'}
{'shopId': '1077891966180733', 'shopName': '中业爱民百货'}
{'shopId': '899526269397757', 'shopName': 'Vango便利店（富通城店）'}
{'shopId': '952839698396961', 'shopName': '农夫山泉安乐特许加盟店'

{'shopId': '1053999063166277', 'shopName': '金都名店品牌男装'}
{'shopId': '1041109866297314', 'shopName': '林记怡宝宝安观澜湖水店（坪洲店）'}
{'shopId': '919996083504449', 'shopName': '吉达桶装水'}
{'shopId': '857066222658588', 'shopName': '宝信商行'}
{'shopId': '959075990900204', 'shopName': '维客佳（宝兴花园店）'}
{'shopId': '994157283831386', 'shopName': '美宜佳（滨城西岸粤18803店）'}
{'shopId': '1095557166682460', 'shopName': '夏记干货'}
{'shopId': '1053917458792392', 'shopName': '名创优品（西乡港隆城广场店）'}
{'shopId': '1080151119025547', 'shopName': 'cookie家宠物俱乐部'}
{'shopId': '1004456615372947', 'shopName': '小酌啤酒体验馆'}
{'shopId': '1022435348472365', 'shopName': '天虹微喔（健兴科技大厦店）'}
{'shopId': '985391255552216', 'shopName': '天虹微喔（富悦公寓店）'}
{'shopId': '851933736731978', 'shopName': '佳源水业'}
{'shopId': '1044842192837571', 'shopName': '张新发槟榔（深圳西丽店）'}
{'shopId': '1002107268238755', 'shopName': '壹号便利（南蓉店）'}
{'shopId': '979189322797592', 'shopName': '东北大板雪糕'}
{'shopId': '1090540644870031', 'shopName': '哈哈镜（宝安店）'}
{'shopId': '864075609301984', 'shopName': '十万大山桶装

...

{'shopId': '946861103928804', 'shopName': '葡萄园进口酒类连锁超市(宝安店)'}
{'shopId': '979077653657367', 'shopName': '德国啤酒专卖店'}
{'shopId': '1098993140555779', 'shopName': '吉米便利店（南头店）'}
{'shopId': '970814136562911', 'shopName': '良品铺子（深圳南山龙海家园店）'}
{'shopId': '985391255552216', 'shopName': '天虹微喔（富悦公寓店）'}
{'shopId': '862928853069176', 'shopName': '丰兴桶装水配送站'}
{'shopId': '957950709481804', 'shopName': '润之家（翻身路店）'}
{'shopId': '851933736731978', 'shopName': '佳源水业'}
{'shopId': '1044842192837571', 'shopName': '张新发槟榔（深圳西丽店）'}
{'shopId': '1046723388548815', 'shopName': '美宜佳（名优采购7855店）'}
{'shopId': '1007377193146594', 'shopName': '鼎湖山泉，白金直营门店'}
{'shopId': '1008876136707527', 'shopName': '八马茶业（深圳龙光世纪店）'}
{'shopId': '1001033526420843', 'shopName': '贸泽酒类直供'}
{'shopId': '1071101622905545', 'shopName': 'Gobuy(径贝店)'}
{'shopId': '852809910088772', 'shopName': '名创优品（宝安河西路店）'}
{'shopId': '880641298143410', 'shopName': '酒小二(翻身店)'}
{'shopId': '1119316925751212', 'shopName': '唯客佳FamilyMart'}
{'shopId': '932868100484322', '

出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误


出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误


出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误
出现错误


ProxyError: HTTPSConnectionPool(host='i.waimai.meituan.com', port=443): Max retries exceeded with url: /openh5/channel/kingkongshoplist?_=1587024517584&X-FOR-WITH=FXdeqbFbEHdtbrfMxUwOVnIdHKB2dewV0gsRUTzXSDnooztjKZhKkJAJHNZzZACCA%2BSO0wvVGmR3fS9z8tETjhV9Z3PjJQpbdFRW3%2FvPVC4g92QFa4RViHPDpqL%2BAkBxRAPL8G0D2ujYxxanLRmAIH0oQzkh%2BI5V99nGUYBsyVhTooUrjL9sxBqAnrs9lYtaQQH0N30SejQcjP70R6408g%3D%3D (Caused by ProxyError('Cannot connect to proxy.', RemoteDisconnected('Remote end closed connection without response')))

# 二.爬取商超的tagId与sku数据

爬取这两个数据不能更改User-Agent，否则页面会出现错误，需要清除缓存后重新登录后才能继续爬虫，其同样也会触发 账号异常然后掉线 的机制

In [14]:
#获取store的分类tagId
def get_spu_tag(sess, store_id): 
    spu_json = get_spuId(sess, store_id)
    spu_tag = []
    if spu_json['msg'] == "成功":
        for s in spu_json['data']['pageCategoryList']:
            spu_tag.append(s['tag'])
           # print(s['tag'])
    return spu_tag

def get_spuId(sess, shop_Id):
    user_agent_list = [
        "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Mobile Safari/537.36",
    ]
    User_Agent = random.choice(user_agent_list)
    headers = {
                "Host": "i.waimai.meituan.com",
                "Cookie": "_lxsdk_cuid=17095e7b964c8-081b2ab2f44e58-5040231b-144000-17095e7b964c8; terminal=i; w_utmz=\"utm_campaign=(direct)&utm_source=5000&utm_medium=(none)&utm_content=(none)&utm_term=(none)\"; w_uuid=d3taEsYmWa5EMBZXi5nkJhrbMmEEcj2t20rktYn7P9vZzLdc286ICBCkZCURxXrv; iuuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; _lxsdk=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; webp=1; __utmz=74597006.1585277189.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; uuid=a36ec623ffdc4ef4b39f.1585293909.1.0.0; wm_order_channel=mtib; a2h=3; ci=328; cityname=%E9%BB%94%E4%B8%9C%E5%8D%97; i_extend=H__a100001__b5; __utma=74597006.2081068397.1585277189.1586310104.1586317204.6; t_lxid=17180a8d90ec3-0f9c0a21015dfb-5040231b-144000-17180a8d90fc8-tid; userId=131673809; token=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; mt_c_token=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; oops=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; w_token=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; utm_source=; channelType={%22mtib%22:%220%22}; _lx_utm=utm_source%3D; channelConfig={%22channel%22:%22default%22%2C%22type%22:0%2C%22fixedReservation%22:{%22reservationTimeStatus%22:0%2C%22startReservationTime%22:0%2C%22endReservationTime%22:0}}; service-off=0; cssVersion=a35856f9; au_trace_key_net=default; openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; w_latlng=0,0; w_actual_lat=26575670; w_actual_lng=107973701; w_visitid=e497c32b-8cb9-461b-9277-bf42f9204a28; _lxsdk_s=171ee012de7-3d6-74e-717%7C%7C58",
                "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
                "Origin": "https://h5.waimai.meituan.com",
                "Accept-Encoding": "br, gzip, deflate",
                "Accept": "application/json",
                "Connection": "keep-alive",

                "Accept-Language": "zh-cn,zh;q=0.9",
                "User-Agent": User_Agent,
            }

    headers_list = headers
    headers_list["Referer"] = "https://h5.waimai.meituan.com/waimai/mindex/menu?dpShopId=&mtShopId=" + str(shop_Id)
    
    formdata = {
                "mtWmPoiId": str(shop_Id),
                " dpShopId": "-1"
            }
    # 超市类的美团外卖链接
    url = "https://i.waimai.meituan.com/openh5/poi/food?_=1588843177748&X-FOR-WITH=j%2FcrK7A2dp7SBa9tlAoUk7uD33L611SZNS5QlwlDxsVTMQkMcXQzr6a7iJV3aPmzZS1sDwjTInFD%2FlpgeJ6zxq%2B2vUIPH67Vqu5dvp6tv%2Fm%2BnprJix2FWxVKLkfKxpygejwiWuXW%2BrdPV3u7Ev%2FF7HqoOdyyAWjG1VQxbEY6uqpGOFErJyhwGmMxovLPk4eXVpWnQ%2BUAwgk8ZbzbMa6nbw%3D%3D"

    proxies = {
                "http": "http://117.158.65.216:50670",
            }

    response = sess.post(url, data=formdata, headers=headers_list)
    detail = response.json()

    
    return detail

In [6]:
#获取store的spu数据
def get_spu_list(ses, store_id, spu_tag, page=0):
    spu_list = pd.DataFrame(columns = ['spuName', 'spuId', 'desc', 'originPrice', 'currentPrice', 'skuList', 'imageUrl'])
    spu_json = get_spu(ses, store_id, spu_tag, page) ##翻页
    time.sleep(random.randint(3,5))
    if spu_json['msg'] == "成功":
        for spu in spu_json['data']['spuList']:
            spu_list = spu_list.append({'spuName':spu['spuName'], 'spuId':spu['spuId'], 'desc': spu['spuDesc'],
                                       'originPrice':spu['originPrice'], 'currentPrice':spu['currentPrice'],
                                       'imageUrl':spu['bigImageUrl'], 'skuList':spu['skuList']}, ignore_index=True)
        if spu_json['data']['end'] is False:
            page += 1
            time.sleep(0.2)
            spu_list = spu_list.append(get_spu_list(ses, store_id, spu_tag, page), ignore_index=True)
    return spu_list

def get_spu(ses, store_id, spu_id, page):
    user_agent_list = [
        "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.79 Mobile Safari/537.36",
    ]
    User_Agent = random.choice(user_agent_list)
    headers = {
                "Host": "i.waimai.meituan.com",
                "Cookie": "_lxsdk_cuid=17095e7b964c8-081b2ab2f44e58-5040231b-144000-17095e7b964c8; terminal=i; w_utmz=\"utm_campaign=(direct)&utm_source=5000&utm_medium=(none)&utm_content=(none)&utm_term=(none)\"; w_uuid=d3taEsYmWa5EMBZXi5nkJhrbMmEEcj2t20rktYn7P9vZzLdc286ICBCkZCURxXrv; iuuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; _lxsdk=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; webp=1; __utmz=74597006.1585277189.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; uuid=a36ec623ffdc4ef4b39f.1585293909.1.0.0; wm_order_channel=mtib; a2h=3; ci=328; cityname=%E9%BB%94%E4%B8%9C%E5%8D%97; i_extend=H__a100001__b5; __utma=74597006.2081068397.1585277189.1586310104.1586317204.6; t_lxid=17180a8d90ec3-0f9c0a21015dfb-5040231b-144000-17180a8d90fc8-tid; userId=131673809; token=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; mt_c_token=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; oops=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; w_token=iYuvMHjCR3zSp-Q4b3yWzmbS41IAAAAAdQoAAPN8gooMqjWJ8MmyyzhbMzh6axtovTtDoSDZYOLvFqAeV1_DHgWEsAvipu9dxMf_vw; utm_source=; channelType={%22mtib%22:%220%22}; _lx_utm=utm_source%3D; channelConfig={%22channel%22:%22default%22%2C%22type%22:0%2C%22fixedReservation%22:{%22reservationTimeStatus%22:0%2C%22startReservationTime%22:0%2C%22endReservationTime%22:0}}; service-off=0; cssVersion=a35856f9; au_trace_key_net=default; openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; w_latlng=0,0; w_actual_lat=26575670; w_actual_lng=107973701; w_visitid=e497c32b-8cb9-461b-9277-bf42f9204a28; _lxsdk_s=171ee012de7-3d6-74e-717%7C%7C58",
                "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
                "Origin": "https://h5.waimai.meituan.com",
                "Accept-Encoding": "br, gzip, deflate",
                "Accept": "application/json",
                "Accept-Language": "zh-cn,zh;q=0.9",
                "User-Agent": User_Agent,
            }

    headers_list = headers
    #headers_list["Referer"] = "https://h5.waimai.meituan.com/waimai/mindex/menu?dpShopId=&mtShopId="+str(store_id) + "&utm_source=60030&channel=mtib&source=shoplist"

    formdata = {
                "wmPoiId": str(store_id),
                "tag": str(spu_id),
                "spuTag": str(spu_id),
                "pageIndex": str(page)
            }
    # 超市类的美团外卖链接
    url = "https://i.waimai.meituan.com/openh5/poi/menuproducts?_=1585894592301&X-FOR-WITH=D9k3yuszKByK7xv%2BziTUYNWGrcNaubLGB2QavEzB5hAIboeZCxySMbPNe2OgPLLIXUofZ3H7N3MGNcDbWnkmEhfGwXAAEj%2Blw2BY9SE%2FzRM4DSLtQJ4d%2FUhQalN7NHNvb%2B3yeoScQRvRv95AMyUGHoaim%2FcpU5QCUR7Z0o3Uw5j2JdURX%2FzsM6BeE%2BRYGhxX72II9aR8mXwTagPw78oBKg%3D%3D"

    # 美食类的美团外卖链接
    # url = 'https://i.waimai.meituan.com/openh5/channel/kingkongshoplist'
    proxies = {
                "http": "http://211.159.219.225:8118",
            }
    
    response = requests.post(url, data=formdata, headers=headers_list, stream=True)
    detail = response.json()
    return detail

In [15]:
sess = requests.Session()
get_spuId(sess, 950640676513559)

{'data': {'outOfRange': 0,
  'mtWmPoiId': '950640676513559',
  'dpShopId': 0,
  'shopStatus': 0,
  'shopPage': True,
  'shopInfo': {'shopName': '诚信平价超市',
   'shopPic': 'http://p0.meituan.net/waimaipoi/b084e344ceef6be52a5e374a1742d704360568.jpg',
   'deliveryFee': 1.0,
   'deliveryType': 0,
   'deliveryTime': 0,
   'deliveryTimeDecoded': '&#xebbe;&#xe513;&#xe513;&#xe513;&#xe3bc;',
   'deliveryMsg': '由 商家 提供配送服务',
   'minFee': 20.0,
   'minFeeDecoded': '&#xeac0;&#xe3bc;.&#xe3bc;',
   'onlinePay': 1,
   'bulletin': '感谢您对诚信平价超市的惠顾！部分商品无法上架，可电话：17876896077.。。。13432084499',
   'discountRestrict': -1,
   'distance': '',
   'bizCode': 0,
   'shipping_time': '&#xe3bc;&#xeab9;:&#xe9e0;&#xe3bc;-&#xeac0;&#xe9e0;:&#xe9e0;&#xe3bc;'},
  'pageCategoryList': [{'tag': '26874899',
    'activityTag': '',
    'iconUrl': '',
    'categoryName': '📍槟榔',
    'categoryType': 0,
    'spuList': [{'spuName': '胖哥 青果庄园青果食用槟榔咖啡味 28g/包',
      'unit': '28克',
      'spuId': 2300533413,
      'tag': '26874899',
      'a

In [ ]:
if __name__ == '__main__':
    spu_name = r'/root/meituan/guangzhou_spu_2000.csv'
    shop_list = pd.read_csv(r'/root/meituan/gz_store.csv')
    
    for index, i in shop_list[2406:2499].iterrows():#获取spuTag
        sess = requests.Session()
        spu_tag = get_spu_tag(sess, int(i['storeId']))
        print(spu_tag, index, int(i['storeId']))
        time.sleep(random.randint(1,2))

        for st in spu_tag[0:]:
            spu_list = get_spu_list(sess, int(i['storeId']), int(st))

            spu_list['shopName'] = i['name']
            spu_list['shopId'] = i['storeId']
            spu_list.to_csv(spu_name, header=0,encoding='utf_8_sig',mode='a+')
            print(spu_list)
            time.sleep(random.random())

# 三. 美团商超地址爬取

        商超地址页面是反爬机制最强的页面，当触发反爬机制时，浏览器将在几天内不能访问此界面，推测他们可能会识别浏览器指纹，故需要多次更换浏览器的User-Agent获取不同的cookie进行爬虫。但这样最多获取5k-1w条数据，之后仍会被检测，被封锁ip几天,想到的解决方案是在代理服务器上登录美团h5页面获取cookie。
        此外，地址的门牌号之类的数据会被加密，会随时间变化不断循环，比如说“1”这个数字对应X套密文，a时间用第一套，b时间用第X套，在c又会回到第一套，且不会发生'#&ed3d'密文对应“1”或“5”，但美团只加密0-9共十个数字，所以进行爬虫时最好记录该时间段的该组密文，便于之后解密

In [2]:
def get_info(store_id):
    user_agent_list = [
                "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
    ]
    User_Agent = random.choice(user_agent_list)
    headers = {
                "Host": "i.waimai.meituan.com",
                "Cookie": "_lxsdk_cuid=17095e7b964c8-081b2ab2f44e58-5040231b-144000-17095e7b964c8; terminal=i; w_utmz=\"utm_campaign=(direct)&utm_source=5000&utm_medium=(none)&utm_content=(none)&utm_term=(none)\"; w_uuid=d3taEsYmWa5EMBZXi5nkJhrbMmEEcj2t20rktYn7P9vZzLdc286ICBCkZCURxXrv; iuuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; _lxsdk=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; webp=1; __utmz=74597006.1585277189.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; uuid=a36ec623ffdc4ef4b39f.1585293909.1.0.0; ci=20; cityname=%E5%B9%BF%E5%B7%9E; wm_order_channel=mtib; utm_source=60030; a2h=3; IJSESSIONID=1fmbae6ndhfc71b77zfzqfz5dk; __utma=74597006.2081068397.1585277189.1585535048.1585990931.4; __utmc=74597006; latlng=26.575229,107.964896,1585990931073; ci3=1; i_extend=H__a100001__b4; au_trace_key_net=default; openh5_uuid=D48AC217DD6D1F7B50B559B22FB814A4C51AA432757A5D8A78C9181DBDA73FE1; service-off=0; w_actual_lat=26576238; w_actual_lng=107977159; w_visitid=fc4c6006-a1f8-435c-aadd-d67a152587ac; w_latlng=23060548,113391350; _lx_utm=utm_source%3D60066; token=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; mt_c_token=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; oops=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; userId=131673809; cssVersion=8c43a663; w_token=T89QJVg6xEnMV_1l_GRnxdvO83sAAAAAWgoAAJR4-GH4JwwMOsBTglnK7DSWgcZ38rZkBvhhbWsl30hWe5pnyGaFQNNE7K0oacbAQA; _lxsdk_s=171531808b6-d6f-da7-e97%7C%7C14; channelType={%22mtib%22:%220%22}",
                "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
                "Origin": "https://h5.waimai.meituan.com",
                "Accept-Encoding": "br, gzip, deflate",
                "Accept": "application/json",
                "Connection": "keep-alive",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-site",
                "Accept-Language": "zh-cn,zh;q=0.9",
                "User-Agent": User_Agent,
            }

    headers_list = headers
    headers_list["Referer"] = "https://h5.waimai.meituan.com/waimai/mindex/menu?dpShopId=&mtShopId=850318828468918&utm_source=&channel=default&source=shoplist"

    formdata = {
                "mtWmPoiId": str(store_id),
            }
    # 超市类的美团外卖链接
    url = "https://i.waimai.meituan.com/openh5/poi/info?_=1586228239797&X-FOR-WITH=Wn7XK5wRPJy9LYsB155gFUBYK2XnTFD12Z13HF4IEHmH1o6gseuyZJTr4XbfawZyZnK7aw7cJc3YxdLhqlcHCrI3YaLk66ewMtBqKpRHhy3DrqsebH58Hr5koP%2BjGscb8hVnIgAK1g90KagNgFWDh8ZElFDTVY7s5xg7Gk1hcJdUX0RY%2Fv"

    # 美食类的美团外卖链接
    # url = 'https://i.waimai.meituan.com/openh5/channel/kingkongshoplist'
    proxies = {
                "http": "http://211.159.219.225:8118",
            }

    response = requests.post(url, data=formdata, headers=headers_list)
    detail = response.json()
    return detail

In [ ]:
shop_list = pd.read_csv(r'/root/meituan/gz_store.csv')
shop_info = r'/root/meituan/guangzhou_shop_info.csv'

for index, i in shop_list[1:].iterrows():#获取spuTag
    info_json = get_info(int(i['storeId']))
    print(index)
    if info_json['msg'] == '成功':
        info_json = info_json['data']
        info = {}
        info['shopName'] = info_json['shopName']
        info['shopAddress'] = info_json['shopAddress']
        info['shopPhone'] = info_json['shopPhone']
        info['shopLat'] = info_json['shopLat']
        info['shopLng'] = info_json['shopLng']
        info['isBrand'] = info_json['isBrand']
        info['shopEnvTitle'] = info_json['shopEnvTitle']
        info['sold'] = info_json['sold']
        pd = pd.DataFrame(info)
        print(pd)
        pd.to_csv(shop_info, encoding='utf_8_sig',mode='a+')
    time.sleep(random.randint(3,5))
    

 全部爬取完成后使用shopId将所有数据合并即可